# Тестирование модулей

## Модуль `datasets`

In [1]:
cd ..

F:\учёба\recs\recs


In [11]:
import dataset

In [12]:
df_video_games = dataset.load_video_games()
df_video_games

,target
0,Wii Sports
1,Super Mario Bros.
2,Mario Kart Wii
3,Wii Sports Resort
4,Pokemon Red/Pokemon Blue
...,...
11557,15 Days
11558,Aiyoku no Eustia
11559,Woody Woodpecker in Crazy Castle 5
11560,LMA Manager 2007


## Модуль `preprocessing`

In [6]:
import preprocessing

In [7]:
cleaner_base = preprocessing.BaseCleaner(remove_number=False)

In [8]:
clear_text = cleaner_base.transform(df_video_games.target)
clear_text[-5:]

['15 days',
 'aiyoku no eustia',
 'woody woodpecker in crazy castle 5',
 'lma manager 2007',
 'haitaka no psychedelica']

## Модуль `models`

In [5]:
import models

ImportError: cannot import name 'Dataset' from 'datasets' (F:\учёба\recs\recs\datasets\__init__.py)

#### TFIDF:

In [8]:
model_tfidf_wrapper = models.TfidfWrapperModel()

In [9]:
%%time
embedding_basedata_tfidf = model_tfidf_wrapper.fit_transform(clear_text)

print(embedding_basedata_tfidf.shape)
embedding_basedata_tfidf

(11562, 9169)
CPU times: total: 250 ms
Wall time: 248 ms


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
# model_tfidf_wrapper.save('testing')

In [11]:
# model_tfidf_wrapper.load('testing')

#### FastText

In [12]:
model_fasttext_wrapper = models.FastTextWrapperModel(
    min_count=3,
    vector_size=100, 
    window=3, 
    sg=1
)

In [13]:
%%time
embedding_basedata_fasttext = model_fasttext_wrapper.fit_transform(clear_text)

print(embedding_basedata_fasttext.shape)
embedding_basedata_fasttext

(11562, 100)
CPU times: total: 4.2 s
Wall time: 3.54 s


array([[-0.14566788, -0.04110384, -0.08632195, ..., -0.07217097,
         0.06976114,  0.04420856],
       [-0.14439343, -0.04345939, -0.0829755 , ..., -0.07839716,
         0.06973555,  0.04046907],
       [-0.14426135, -0.04093863, -0.08609829, ..., -0.07420567,
         0.06972575,  0.04349964],
       ...,
       [-0.14383729, -0.04226118, -0.08653367, ..., -0.07478501,
         0.06843017,  0.04093023],
       [-0.14383869, -0.04326478, -0.08702809, ..., -0.06931728,
         0.07004426,  0.04359977],
       [-0.141986  , -0.04554222, -0.08709615, ..., -0.07256959,
         0.06801727,  0.04175485]], dtype=float32)

## Модуль `similarity_search`

In [33]:
import similarity_search

#### ForCycleSearch

In [17]:
search_for_cycle = similarity_search.ForCycleSearch(model_fasttext_wrapper, embedding_basedata_fasttext, df_video_games.target.values)

In [18]:
%%time
search_for_cycle.search('mario', 5)

CPU times: total: 219 ms
Wall time: 264 ms


,name,similarity
1130,Paper Mario,0.999842
596,Mario Bros.,0.999838
473,Mario Paint,0.999825
2924,Mario & Wario,0.999823
1055,Mario Golf,0.999820


In [19]:
%%time
search_for_cycle.search('mario 9', 5)

CPU times: total: 203 ms
Wall time: 221 ms


,name,similarity
381,Mario Party 9,0.999812
4814,Wild 9,0.999801
9751,Medabots 9: Metabee / Rokusho,0.999726
9799,Medarot 9: Kabuto Ver. / Kuwagata Ver.,0.999675
6918,J-League Winning Eleven 9: Asia Championship,0.999659


In [23]:
%%time
search_for_cycle.search('maio', 5)

CPU times: total: 203 ms
Wall time: 219 ms


,name,similarity
7581,Sidewinder Max,0.999762
5772,Bomberman B-Daman,0.999754
2785,Seaman,0.999753
353,Max Payne,0.999746
9501,Gitaroo Man Lives!,0.999746


## `TEMP` code:

In [2]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
# from datasets import Dataset
from torch.utils.data import Dataset, DataLoader

In [3]:
class CustomDataset(Dataset):

    def __init__(self, array, augmentation=None):
        self.array = [[text, text] for text in array]
        self.augmentation = augmentation

    def __len__(self):
        return len(self.array)

    def __getitem__(self, idx):

        lst = self.array[idx]
        return InputExample(texts=[lst[0], lst[1]])

In [4]:
# array = df_video_games.target.values
# array

In [5]:
# array = [[text, text] for text in array]
# array[:5]

In [6]:
# import pandas as pd
# train_df = pd.DataFrame({'set': array})
# train_df

In [7]:
# dataset = Dataset.from_pandas(train_df)
# dataset

In [8]:
# train_examples = []
# train_data = dataset["set"]
# n_examples = dataset.num_rows

# for i in range(n_examples):
#     example = train_data[i]
#     if example[0] == None: #remove None
#         print(example)
#         continue        
#     train_examples.append(InputExample(texts=[str(example[0]), str(example[1])]))

In [9]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [34]:
from utils import _cosine_distance

Base

In [35]:
array1 = model.encode('maio')
array2 = model.encode('Super Mario Bros.')
_cosine_distance(array1, array2)

0.10861024720662833

In [36]:
array1 = model.encode('maio')
array2 = model.encode('Mario')
_cosine_distance(array1, array2)

0.2950194479607329

In [48]:
array1 = model.encode('Max 5')
array2 = model.encode('Maxim 5')
_cosine_distance(array1, array2)

0.8449747411830386

In [49]:
array1 = model.encode('Мария')
array2 = model.encode('Маия')
_cosine_distance(array1, array2)

0.6417559815496462

Tuning

In [13]:
train_examples = CustomDataset(df_video_games.target.values)

In [14]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
# num_epochs = 10
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [15]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/181 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [52]:
array1 = model.encode('maio')
array2 = model.encode('Super Mario Bros.')
_cosine_distance(array1, array2)

-0.09821634156992119

In [53]:
array1 = model.encode('maio')
array2 = model.encode('Mario')
_cosine_distance(array1, array2)

0.07148838958792078

In [54]:
array1 = model.encode('Max 5')
array2 = model.encode('Maxim 5')
_cosine_distance(array1, array2)

0.8812535528163306

In [55]:
array1 = model.encode('Мария')
array2 = model.encode('Маия')
_cosine_distance(array1, array2)

0.529520837670178